In [20]:
from datetime import datetime
import requests

In [23]:
file_uri = '/path/to/file'
mask1 = {'fill': 'toself', 'x': [0, 1, 2, 0], 'y': [0, 2, 0, 0], 'type': 'scatter'}
mask2 = {'fill': 'toself', 'x': [3,3,5,5,3], 'y': [0.5,1.5,1.5,0.5,0.5], 'type': 'scatter'}
SPLASH_URI = 'http://localhost:8087/api/v0'

### Posting a new tagging event for version control

In [24]:
new_tagging_event = {
    'tagger_id': 'username',
    'run_time': str(datetime.utcnow())
}
response = requests.post(f'{SPLASH_URI}/events', json=new_tagging_event)
event_id = response.json()['uid']

### Posting a data set with a set of tags

In [31]:
dataset = {
    'uri': file_uri,
    'type': 'file',
    'project': 'project_id',    # Project ID groups a set of datasets
    'tags': [
        {'name': 'class_name_1',
         'locator': {
             'spec': 'plotly_annotation',
             'path': mask1
         },
         'event_id': event_id},
        {'name': 'class_name_2',
         'locator': {
             'spec': 'plotly_annotation',
             'path': mask2
         },
         'event_id': event_id,
        }
    ]
}

response = requests.post(f'{SPLASH_URI}/datasets', json=dataset)
status = response.status_code
dataset_uid = response.json()['uid']
print(f'Status code: {status}, dataset uid: {dataset_uid}')

Status code: 200, dataset uid: 20d6e416-fe40-4353-96c8-3dd815b00955


### Quering datasets

In [33]:
response = requests.get(f'{SPLASH_URI}/datasets', 
                        params={"page[offset]": 0,      # offset for query
                                "page[limit]": 10,      # length of queried items
                                "uri": file_uri,
                                "event_id": event_id})
splash_dataset = response.json()

print(f'splash_dataset retrieved with query: {splash_dataset}')

splash_dataset retrieved with query: [{'uid': '20d6e416-fe40-4353-96c8-3dd815b00955', 'schema_version': '1.2', 'project': 'project_id', 'type': 'file', 'uri': '/path/to/file', 'tags': [{'uid': '2a3d7431-cd2d-47ca-94e7-ff547351815e', 'name': 'class_name_1', 'locator': {'spec': 'plotly_annotation', 'path': {'fill': 'toself', 'x': [0, 1, 2, 0], 'y': [0, 2, 0, 0], 'type': 'scatter'}}, 'confidence': None, 'event_id': 'c0eb1f88-490d-4553-8c6d-ba00d2cfffb2'}, {'uid': 'b98ea082-6431-415c-bea2-dcb0c0a05edc', 'name': 'class_name_2', 'locator': {'spec': 'plotly_annotation', 'path': {'fill': 'toself', 'x': [3, 3, 5, 5, 3], 'y': [0.5, 1.5, 1.5, 0.5, 0.5], 'type': 'scatter'}}, 'confidence': None, 'event_id': 'c0eb1f88-490d-4553-8c6d-ba00d2cfffb2'}]}]


### New tagging event with new version of tags

In [34]:
mask3 = {'fill': 'toself', 'x': [3,3,5,1,3], 'y': [0.5,1,1.5,0.5,0.5], 'type': 'scatter'}

new_tagging_event = {
    'tagger_id': 'username',
    'run_time': str(datetime.utcnow())
}
response = requests.post(f'{SPLASH_URI}/events', json=new_tagging_event)
event_id2 = response.json()['uid']

### Add new tag to existing dataset

In [37]:
new_tags = [
    {'name': 'class_name_3',
     'event_id': event_id2,
     'locator': {
         'spec': 'plotly_annotation',
         'path': mask3
     }}
]

response = requests.patch(f'{SPLASH_URI}/datasets/{dataset_uid}/tags', 
                          json={'add_tags': new_tags})
status = response.status_code
tags_uids = response.json()
print(f'Status code: {status}, the uids of the added tags are: {tags_uids["added_tags_uid"]}')

Status code: 200, the uids of the added tags are: ['dbca59a2-8db5-4a41-b6b8-91affdedcc41']
